# Azure AI Agents with Model Context Protocol (MCP) Support - Python

এই নোটবুকটি দেখায় কীভাবে Python-এ Model Context Protocol (MCP) টুল সহ Azure AI Agents ব্যবহার করতে হয়। এটি দেখায় কীভাবে একটি বুদ্ধিমান এজেন্ট তৈরি করা যায় যা বাহ্যিক MCP সার্ভার (যেমন Microsoft Learn) ব্যবহার করে উন্নত ক্ষমতা অর্জন করতে পারে, যেখানে কীলেস প্রমাণীকরণ ব্যবহার করা হয়।


## প্রয়োজনীয় পাইথন প্যাকেজ ইনস্টল করুন

প্রথমে, আমাদের প্রয়োজনীয় পাইথন প্যাকেজগুলো ইনস্টল করতে হবে:
- **azure-ai-projects**: মূল Azure AI Projects SDK
- **azure-ai-agents**: এজেন্ট তৈরি এবং পরিচালনার জন্য Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential ব্যবহার করে কী-ছাড়া প্রমাণীকরণ প্রদান করে
- **mcp**: পাইথনের জন্য Model Context Protocol বাস্তবায়ন


## কীলেস অথেন্টিকেশন এর সুবিধা

এই নোটবুকটি **কীলেস অথেন্টিকেশন** প্রদর্শন করে, যা বেশ কিছু সুবিধা প্রদান করে:
- ✅ **কোনো API কী পরিচালনা করার প্রয়োজন নেই** - Azure এর পরিচয়-ভিত্তিক অথেন্টিকেশন ব্যবহার করে
- ✅ **উন্নত নিরাপত্তা** - কোড বা কনফিগারেশন ফাইলে কোনো গোপন তথ্য সংরক্ষণ করা হয় না
- ✅ **স্বয়ংক্রিয় ক্রেডেনশিয়াল রোটেশন** - Azure ক্রেডেনশিয়াল লাইফসাইকেল ম্যানেজমেন্ট পরিচালনা করে
- ✅ **রোল-ভিত্তিক অ্যাক্সেস কন্ট্রোল** - সূক্ষ্ম অনুমতির জন্য Azure RBAC ব্যবহার করে
- ✅ **মাল্টি-এনভায়রনমেন্ট সাপোর্ট** - ডেভেলপমেন্ট এবং প্রোডাকশন উভয় ক্ষেত্রেই নির্বিঘ্নে কাজ করে

`DefaultAzureCredential` স্বয়ংক্রিয়ভাবে সেরা উপলব্ধ ক্রেডেনশিয়াল উৎস নির্বাচন করে:
1. **Managed Identity** (যখন Azure-এ চলছে)
2. **Azure CLI** ক্রেডেনশিয়াল (লোকাল ডেভেলপমেন্টের সময়)
3. **Visual Studio** ক্রেডেনশিয়াল
4. **Environment variables** (যদি কনফিগার করা থাকে)
5. **ইন্টারঅ্যাকটিভ ব্রাউজার** অথেন্টিকেশন (ফলব্যাক হিসেবে)


## কীলেস প্রমাণীকরণ সেটআপ

**কীলেস প্রমাণীকরণের জন্য প্রয়োজনীয় শর্তাবলী:**

### স্থানীয় ডেভেলপমেন্টের জন্য:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure পরিবেশের জন্য:
- আপনার Azure রিসোর্সে **System-assigned Managed Identity** সক্রিয় করুন
- ম্যানেজড আইডেন্টিটিকে উপযুক্ত **RBAC ভূমিকা** প্রদান করুন:
  - Azure OpenAI অ্যাক্সেসের জন্য `Cognitive Services OpenAI User`
  - Azure AI Projects অ্যাক্সেসের জন্য `AI Developer`

### পরিবেশ ভেরিয়েবল (ঐচ্ছিক):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**কোনো API কী বা সংযোগ স্ট্রিংয়ের প্রয়োজন নেই!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## প্রয়োজনীয় লাইব্রেরি আমদানি করুন

প্রয়োজনীয় পাইথন মডিউলগুলো আমদানি করুন:
- **os, time**: পরিবেশ ভেরিয়েবল এবং বিলম্বের জন্য স্ট্যান্ডার্ড পাইথন লাইব্রেরি
- **AIProjectClient**: Azure AI Projects-এর জন্য প্রধান ক্লায়েন্ট
- **DefaultAzureCredential**: Azure পরিষেবাগুলোর জন্য কী ছাড়া প্রমাণীকরণ
- **MCP-সম্পর্কিত ক্লাসসমূহ**: MCP টুল তৈরি ও পরিচালনা এবং অনুমোদন পরিচালনার জন্য


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP সার্ভার সেটিংস কনফিগার করুন

পরিবেশ ভেরিয়েবল ব্যবহার করে MCP সার্ভারের কনফিগারেশন সেট আপ করুন, যেখানে ডিফল্ট মান থাকবে:
- **MCP_SERVER_URL**: MCP সার্ভারের URL (ডিফল্ট Microsoft Learn API)
- **MCP_SERVER_LABEL**: MCP সার্ভার চিহ্নিত করার জন্য একটি লেবেল (ডিফল্ট "mslearn")

এই পদ্ধতি বিভিন্ন পরিবেশে নমনীয় কনফিগারেশন নিশ্চিত করে।


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI প্রকল্প ক্লায়েন্ট তৈরি করুন (কী ছাড়া প্রমাণীকরণ)

**কী ছাড়া প্রমাণীকরণ** ব্যবহার করে Azure AI প্রকল্প ক্লায়েন্ট আরম্ভ করুন:
- **endpoint**: Azure AI Foundry প্রকল্পের endpoint URL
- **credential**: নিরাপদ, কী ছাড়া প্রমাণীকরণের জন্য `DefaultAzureCredential()` ব্যবহার করে
- **API কী প্রয়োজন নেই**: স্বয়ংক্রিয়ভাবে সেরা উপলব্ধ credential খুঁজে বের করে এবং ব্যবহার করে

**প্রমাণীকরণ প্রবাহ:**
1. Managed Identity (Azure পরিবেশে) পরীক্ষা করে
2. স্থানীয় ডেভেলপমেন্টের জন্য Azure CLI credentials-এ ফিরে যায়
3. প্রয়োজন অনুযায়ী অন্যান্য উপলব্ধ credential উৎস ব্যবহার করে

এই পদ্ধতি আপনার কোডে API কী বা সংযোগ স্ট্রিং পরিচালনার প্রয়োজনীয়তা দূর করে।


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP টুল সংজ্ঞা তৈরি করুন

একটি MCP টুল তৈরি করুন যা Microsoft Learn MCP সার্ভারের সাথে সংযোগ স্থাপন করে:
- **server_label**: MCP সার্ভারের পরিচয়
- **server_url**: MCP সার্ভারের URL এন্ডপয়েন্ট
- **allowed_tools**: ঐচ্ছিক তালিকা যা নির্দিষ্ট টুল ব্যবহারে সীমাবদ্ধ করে (খালি তালিকা সব টুল ব্যবহারের অনুমতি দেয়)

এই টুলটি এজেন্টকে Microsoft Learn ডকুমেন্টেশন এবং রিসোর্সে প্রবেশের সুযোগ দেবে।


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## এজেন্ট তৈরি এবং কথোপকথন সম্পাদন (কীলেস ওয়ার্কফ্লো)

এই বিস্তৃত অংশটি সম্পূর্ণ **কীলেস এজেন্ট ওয়ার্কফ্লো** প্রদর্শন করে:

1. **এআই এজেন্ট তৈরি করুন**: GPT-4.1 ন্যানো মডেল এবং MCP টুলস দিয়ে একটি এজেন্ট সেট আপ করুন  
2. **থ্রেড তৈরি করুন**: যোগাযোগের জন্য একটি কথোপকথন থ্রেড স্থাপন করুন  
3. **বার্তা পাঠান**: এজেন্টকে Azure OpenAI এবং OpenAI এর পার্থক্য সম্পর্কে জিজ্ঞাসা করুন  
4. **টুল অনুমোদন পরিচালনা করুন**: প্রয়োজন হলে MCP টুল কল স্বয়ংক্রিয়ভাবে অনুমোদন করুন  
5. **কার্যক্রম পর্যবেক্ষণ করুন**: এজেন্টের অগ্রগতি ট্র্যাক করুন এবং প্রয়োজনীয় কোনো পদক্ষেপ নিন  
6. **ফলাফল প্রদর্শন করুন**: কথোপকথন এবং টুল ব্যবহারের বিবরণ দেখান  

**কীলেস বৈশিষ্ট্যসমূহ:**
- ✅ **কোনো হার্ডকোডেড সিক্রেট নেই** - সমস্ত প্রমাণীকরণ Azure পরিচয় দ্বারা পরিচালিত  
- ✅ **ডিফল্টভাবে সুরক্ষিত** - ভূমিকা-ভিত্তিক অ্যাক্সেস নিয়ন্ত্রণ ব্যবহার করে  
- ✅ **সহজ ডিপ্লয়মেন্ট** - কোনো ক্রেডেনশিয়াল ব্যবস্থাপনার প্রয়োজন নেই  
- ✅ **অডিট-বান্ধব** - সমস্ত অ্যাক্সেস Azure পরিচয়ের মাধ্যমে ট্র্যাক করা হয়  

এজেন্ট MCP টুলস ব্যবহার করে Microsoft Learn রিসোর্সে সম্পূর্ণ নিরাপত্তার সাথে এবং কোনো API কী ব্যবস্থাপনা ছাড়াই অ্যাক্সেস করবে।


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
